# Question 2

In [ ]:
import numpy as np

w1 = 0.10193627
w2 = 0.20322712
w3 = 0.30232353
w4 = 0.4038725
w5 = 0.5083905
w6 = 0.6187173

h1 = w1*3 + w2*5
h2 = w3*3 + w4*5

yhat = 1.0 / (1.0 + np.exp(-h1*w5 - h2*w6))
constant = 2*(yhat - 1) * yhat * (1-yhat)
yhat

0.9229156017085289

In [ ]:
gradient = constant * w5 * 3
gradient

-0.016727985296864145

In [ ]:
w1 - 0.1*gradient

0.1036090685296864

# Question 3

First of all, we import pytorch and some requirements to implement a model on CIFAR10 dataset.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as functional
from torchvision import transforms 
from torchvision.datasets import CIFAR10
from torch.nn import Linear, ReLU, Conv2d, MaxPool2d, BatchNorm2d, Dropout2d, Dropout

Now we have to define our convolutional neural network and implement the layers according to the assignment's documentation.

In [3]:
class Convolutional_network(nn.Module):
  def __init__(self):
    super(Convolutional_network, self).__init__()
    self.cnn_layers = nn.Sequential(
          # Defining a 2D convolution layer
          Conv2d(in_channels=3, out_channels=4, kernel_size=3, stride=1, padding=1),
          BatchNorm2d(4),
          ReLU(inplace=True),
          # Defining another 2D convolution layer
          Conv2d(in_channels=4, out_channels=8, kernel_size=3, stride=1, padding=1),
          ReLU(inplace=True),
          MaxPool2d(kernel_size=2, stride=2),
          # Defining another 2D convolution layer
          Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
          BatchNorm2d(16),
          ReLU(inplace=True),
          # Defining another 2D convolution layer
          Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
          ReLU(inplace=True),
          MaxPool2d(kernel_size=2, stride=2),

          Dropout2d(0.05),
          # Defining another 2D convolution layer
          Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
          BatchNorm2d(64),
          ReLU(inplace=True),
          # Defining another 2D convolution layer
          Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
          ReLU(inplace=True),
          MaxPool2d(kernel_size=2, stride=2),
      )

    self.linear_layers = nn.Sequential(
          Dropout2d(0.05),
          Linear(2048, 512),
          ReLU(inplace=True),
          Linear(512, 128),
          ReLU(inplace=True),
          Dropout(0.05),
          Linear(128, 10),
      )

  # Defining the forward pass    
  def forward(self, x):
      x = self.cnn_layers(x)
      x = x.view(x.size(0), -1)
      x = self.linear_layers(x)
      return x

Hyperparameters are so important for any neural network, we initialize them in this cell before we start to train the model.

In [4]:
model = Convolutional_network()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
batch_size = 4
epochs = 2
data_augmentation = False

For the extra point, we may need to augment some data. We can do this using by adding random rotation or random cropping to our images of the dataset, As you see in the next cell.

In [5]:
if data_augmentation:
  transform = transforms.Compose(
    [transforms.RandomAutocontrast(),
     transforms.RandomResizedCrop(32),
     transforms.RandomRotation(45),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
else:
  transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

Then we load the dataset as you explained on the assignment. We save the dataset on trainloader and testloader variables.

In [6]:
trainset = CIFAR10(root='./data', train=True,
                   download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
classes = trainset.classes

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


This is trainig cell for our model. As we know it has forward propagation, calculate loss, backward propagation and optimize the weights and biases. Each step has been done using torch implemented functions.

In [7]:
for epoch in range(int(epochs)):
  for images, labels in trainloader:
    optimizer.zero_grad()
    outputs = model(images)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


To see the train accuracy for each of the classes you can run the next cell.

In [8]:
prediction_and_total = {classname: [0, 0] for classname in classes}

for images, labels in trainloader:
  outputs = model(images)
  _, predictions = torch.max(outputs, 1)
  
  for label, prediction in zip(labels, predictions):
    if label == prediction:
      prediction_and_total[classes[label]][0] += 1
    prediction_and_total[classes[label]][1]+=1

for classname, result in prediction_and_total.items():
  accuracy = 100 * float(result[0]) / float(result[1])
  print(classname, "train accuracy:", accuracy)

airplane train accuracy: 77.42
automobile train accuracy: 79.88
bird train accuracy: 41.32
cat train accuracy: 42.0
deer train accuracy: 41.78
dog train accuracy: 48.1
frog train accuracy: 59.86
horse train accuracy: 68.34
ship train accuracy: 50.84
truck train accuracy: 67.16


Finally we can test our model on the test set and see the accuracy for each class using the following code.

In [9]:
prediction_and_total = {classname: [0, 0] for classname in classes}

for images, labels in testloader:
  outputs = model(images)
  _, predictions = torch.max(outputs, 1)
  
  for label, prediction in zip(labels, predictions):
    if label == prediction:
      prediction_and_total[classes[label]][0] += 1
    prediction_and_total[classes[label]][1]+=1

for classname, result in prediction_and_total.items():
  accuracy = 100 * float(result[0]) / float(result[1])
  print(classname, "test accuracy:", accuracy)

airplane test accuracy: 77.7
automobile test accuracy: 75.6
bird test accuracy: 40.6
cat test accuracy: 38.4
deer test accuracy: 40.4
dog test accuracy: 48.0
frog test accuracy: 60.4
horse test accuracy: 65.9
ship test accuracy: 45.9
truck test accuracy: 65.5
